In [ ]:
!pip install mosestokenizer

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for mosestokenizer: filename=mosestokenizer-1.2.1-py3-none-any.whl size=49170 sha256=d2b9259503cab19af0f03aa21151f458caeb024accdcf51dab9964b8907348ad
  Stored in directory: /root/.cache/pip/wheels/80/d8/15/4c5ebbe883513f003cb055a0369c77c9df857023a706f39e70
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6e88a4bcb72878d0ec1bd69980edcfed5a91cbe2b549ed0df34c9f6a3c7b3db3
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3338 sha256=bd12865f20fc651bef1b864b6b56285c1f502e0c334c2e0824ad21b8975b9482
  Stored in directory: /root/.cache/pip/wheels/e1/af/b1/99b57a06dda78fdcee86d2e22c64743f3b8df8f31cfc04baf7
  Created wheel for uctools: filenam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Assuming that a folder named `6809_a1_data` is created and contains the CSV file.

In [ ]:
import pandas as pd
from collections import Counter
from mosestokenizer import *
from tqdm import tqdm
import string
import os, json

In [ ]:
data_root = "/content/drive/MyDrive/6809_a1_data"
csv_path = os.path.join(data_root, "goodreads_data.csv")
english_sw = os.path.join(data_root, "english")

In [ ]:
df = pd.read_csv(csv_path)

In [ ]:
df

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...
...,...,...,...,...,...,...,...,...
9995,9995,"Breeders (Breeders Trilogy, #1)",Ashley Quigley,How far would you go? If human society was gen...,"['Dystopia', 'Science Fiction', 'Post Apocalyp...",3.44,276,https://www.goodreads.com/book/show/22085400-b...
9996,9996,Dynamo,Eleanor Gustafson,Jeth Cavanaugh is searching for a new life alo...,[],4.23,60,https://www.goodreads.com/book/show/20862902-d...
9997,9997,The Republic of Trees,Sam Taylor,This dark fable tells the story of four Englis...,"['Fiction', 'Horror', 'Dystopia', 'Coming Of A...",3.29,383,https://www.goodreads.com/book/show/891262.The...
9998,9998,"Waking Up (Healing Hearts, #1)",Renee Dyer,For Adriana Monroe life couldn’t get any bette...,"['New Adult', 'Romance', 'Contemporary Romance...",4.13,263,https://www.goodreads.com/book/show/19347252-w...


In [ ]:
genres = set()

In [ ]:
def transform_genre_to_lst(txt):
  return txt.strip('][').replace("'", "").split(', ')

In [ ]:
genres_col = "Genres"

In [ ]:
for text in tqdm(df[genres_col]):
  _genres = transform_genre_to_lst(text)
  genres.update(_genres)

100%|██████████| 10000/10000 [00:00<00:00, 115135.45it/s]


In [ ]:
len(genres)

618

In [ ]:
genres2idx= {}

In [ ]:
for idx, g in enumerate(genres):
  genres2idx[g] = idx

In [ ]:
# Keeping stopwords because they can potentially appear in autcompletion.
# E.g: Bob ate -> a, an ect.

In [ ]:
def read_stopwords(file_path):
    """
    Reads a list of stopwords from a file. Each line in the file should contain one stopword.

    :param file_path: The path to the file containing the stopwords.
    :return: A list of stopwords.
    """
    stopwords = []
    try:
        with open(file_path, 'r') as file:
            stopwords = [line.strip() for line in file.readlines()]
    except FileNotFoundError:
        print(f"The file {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return stopwords

In [ ]:
stop_words = read_stopwords(english_sw)

In [ ]:
# Initial vocab containing stopwords
vocab = {i:w for i, w in enumerate(stop_words)}

In [ ]:
len(vocab)

179

In [ ]:
remaining_words = 5000 - len(vocab) - 1 # -1 for the UNK token that will added in the end.

In [ ]:
def remove_punctuation_whitespace(text):
    """Remove punctuation and common whitespace characters from a piece of text."""
    # Define the punctuation and whitespace characters to remove
    punctuation_and_whitespace = string.punctuation + '\t\n\r'

    if not isinstance(text, str):
      text = str(text)

    # Create a translation table that maps each character to None
    translator = str.maketrans('', '',punctuation_and_whitespace)

    # Translate the text using the translation table
    return text.translate(translator)

In [ ]:
def build_vocab(df, text_column, stop_words, num_words=5000):
    # Initialize the tokenizer
    tokenizer = MosesTokenizer()

    # Tokenize each row and collect tokens in a list
    all_tokens = []
    for text in tqdm(df[text_column]):
        text = remove_punctuation_whitespace(text).lower()
        tokens = tokenizer(text)
        all_tokens.extend(tokens)

    # Count the frequency of each token
    word_freq = Counter(all_tokens)

    # Filter out the stop words and select the top num_words
    filtered_words = [word for word in word_freq if word.lower() not in stop_words]
    top_words = dict(Counter(filtered_words).most_common(num_words))

    # Clean up the tokenizer to free memory
    tokenizer.close()

    return list(top_words.keys())

In [ ]:
# Reserve last 1k and second to last 1k for test and dev.
# Select the last 1000 rows for test
test_df = df.iloc[-1000:]
# Select the second last 1000 rows for development
dev_df = df.iloc[-2000:-1000]
# Select everything up to the second last 1000 rows for training
train_df = df.iloc[:-2000]

In [ ]:
test_df.to_csv(os.path.join(data_root, "test.csv"), index=None)
dev_df.to_csv(os.path.join(data_root, "dev.csv"), index=None)

In [ ]:
remaining_vocab = build_vocab(df, "Description", stop_words, remaining_words)

100%|██████████| 10000/10000 [00:04<00:00, 2370.11it/s]


In [ ]:
assert len(remaining_vocab) == remaining_words

In [ ]:
range(len(vocab), len(remaining_vocab) + len(vocab))

range(178, 4999)

In [ ]:
end_idx_vocab = len(vocab) + 1

In [ ]:
_vocab = {i: remaining_vocab[end_idx_vocab - i] for i in range(len(vocab), len(remaining_vocab) + len(vocab))}

In [ ]:
final_vocab =  {**vocab, **_vocab}

In [ ]:
len(final_vocab)

4999

In [ ]:
vocab2idx = {v:k for k,v in final_vocab.items()}

In [ ]:
len(vocab2idx)

4999

In [ ]:
vocab2idx['<unk>'] = 5000

In [ ]:
len(vocab2idx)

5000

In [ ]:
def save_dict_as_json(data, file_path):
    """Save a dictionary as a JSON file.

    Args:
        data (dict): The dictionary to save.
        file_path (str): The path to the file where to save the JSON data.
    """
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
save_dict_as_json(final_vocab, os.path.join(data_root, "vocab_2.json"))

In [ ]:
save_dict_as_json(vocab2idx, os.path.join(data_root, "vocab2idx_2.json"))

In [ ]:
save_dict_as_json(genres2idx, os.path.join(data_root, "genres2idx.json"))

In [ ]:
# Now will generate actual data to be fed to the model
# We will create a tuple(list, int), were `list` contains a set of word ids from 1..n and `int` is n+1th word.

In [ ]:
ctx_len = 20

In [ ]:
def create_ctx(lst, ctx_len):
  pairs = []
  for i in range(len(lst) - ctx_len):
    ctx = lst[i: ctx_len + i]
    next_word = lst[i + ctx_len]
    pairs.append((ctx, next_word))

  return pairs

In [ ]:
def build_pairs(df, text_column, vocab2idx, genres2idx, ctx_len=5):

  tokenizer = MosesTokenizer()
  all_pairs = []
  genres_num = len(genres2idx)

  for text in tqdm(df.itertuples()):
      sample_genres = transform_genre_to_lst(text.Genres)
      sample_genres_ids = list(map(lambda x: genres2idx[x], sample_genres))
      sample_text = remove_punctuation_whitespace(text.Description).lower()
      tokens = tokenizer(sample_text)
      token_ids = [vocab2idx[t] if t in vocab2idx else 5000 for t in tokens]
      pairs = create_ctx(token_ids, ctx_len)
      for g_id in sample_genres_ids:
        for pair in pairs:
          new_pair = (pair) + (g_id, )
          all_pairs.append(new_pair)

  return all_pairs

In [ ]:
training_pairs = build_pairs(train_df, "Description", vocab2idx, genres2idx, ctx_len)

8000it [00:08, 955.54it/s]


In [ ]:
len(training_pairs)

6926634

In [ ]:
testing_pairs = build_pairs(test_df, "Description", vocab2idx, genres2idx, ctx_len)

1000it [00:00, 1699.06it/s]


In [ ]:
dev_pairs = build_pairs(dev_df, "Description", vocab2idx, genres2idx, ctx_len)

1000it [00:00, 1682.58it/s]


In [ ]:
len(testing_pairs)

486880

In [ ]:
len(dev_pairs)

789217

Saving the dataset files for later use if needed.

In [ ]:
import pickle

def save_list_as_pickle(data, file_path):
    """Save a list as a pickle file.

    Args:
        data (list): The list to save.
        file_path (str): The path to the file where to save the pickle data.
    """
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

In [ ]:
save_list_as_pickle(training_pairs, os.path.join(data_root, "train_2.pkl"))
save_list_as_pickle(testing_pairs, os.path.join(data_root, "test_2.pkl"))
save_list_as_pickle(dev_pairs, os.path.join(data_root, "dev_2.pkl"))